# Introduction to data wrangling using _`pandas`_.

### Import packages

In [1]:
#data processing
import pandas as pd

#plotting
import matplotlib.pyplot as plt
import seaborn as sns

### Set-up

For this expercise, we will utilize two files (listed below) from the 'Brazilian E-Commerce Public Dataset' originally posted on [Kaggle]( https://www.kaggle.com/olistbr/brazilian-ecommerce).

In [2]:
#also see kaggle (above)

# full path of the 'orders' dataset
orders_file = 'https://bitbucket.org/vishal_derive/vcu-data-mining/raw/11253d9f443241b3ce5949802966a80de73af1db/data/olist_orders_dataset.csv'

# full path of the 'customer' dataset
cust_file = 'https://bitbucket.org/vishal_derive/vcu-data-mining/raw/11253d9f443241b3ce5949802966a80de73af1db/data/olist_customers_dataset.csv'


### Read data

In [3]:
orders = pd.read_csv(orders_file) #99K records/8 columns

orders.shape

(99441, 8)

In [4]:
# let's take a look at order_status 

orders.head()

orders.order_status.value_counts() #or orders['order_status'].value_counts()

orders.order_status.value_counts()/len(orders) #percentage

delivered      0.970203
shipped        0.011132
canceled       0.006285
unavailable    0.006124
invoiced       0.003158
processing     0.003027
created        0.000050
approved       0.000020
Name: order_status, dtype: float64

For this exercise, we will use only four columns from this data frame: `order_id`, `customer_id`, `order_status`, `order_purchase_date`. Let's drop the other columns.

In [9]:
# drop unnecessary columns

drop_vars = ['order_approved_at', 'order_delivered_carrier_date', 
             'order_delivered_customer_date', 'order_estimated_delivery_date']

orders = orders.drop(drop_vars, axis=1)

KeyError: "['order_approved_at' 'order_delivered_carrier_date'\n 'order_delivered_customer_date' 'order_estimated_delivery_date'] not found in axis"

In [ ]:
# check column types



### Convert `order_purchase_timestamp` from `object` to `datetime`

In [ ]:
# date-time conversion

orders['order_purchase_timestamp'] = 

orders.dtypes

In [ ]:
# for this exercise, we won't use the time-stamp
# let's convert the order purchase date-timestamps into dates

orders['order_purchase_date'] = 

In [ ]:
# view these two columns
orders[['order_purchase_timestamp', 'order_purchase_date']].head()

### Daily order volumes

In [ ]:
# group by order date and count the number of orders (for each date)

order_counts = 

order_counts.head()

The `groupby()` function with a summary function like `count()` returns an indexed pandas dataframe. 

_`pandas`_ uses a similar structure like **SQL** for aggregating data. Here's an analogous SQL query:

`CREATE TABLE order_counts as`

`SELECT order_purchase_date, count(order_id)`

`FROM orders`

`GROUP BY order_purchase_date;`

Plot the daily order volume.

In [ ]:
# set style
sns.set(style='darkgrid')

# adjust the plot size
plt.figure(figsize=[15, 6])

# line plot
sns.lineplot()

# assign axis labels
plt.ylabel('Number of Orders', fontsize=14)
plt.xlabel('Order Purchase Date', fontsize=14);

Some of the recent data seem to be missing or incomplete. Let's take a closer look.

In [ ]:
import matplotlib.dates as mdates

# set style
sns.set(style='darkgrid')

# adjust the plot size
fig, ax = plt.subplots(figsize=[15, 6])

# line plot
sns.lineplot(x=order_counts.index, 
             y=order_counts.values, 
             marker='s', 
             color='royalblue')

# assign axis labels
plt.ylabel('Number of Orders', fontsize=14)
plt.xlabel('Order Purchase Date', fontsize=14)

ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d-%y'));

It appears that perhaps 8/22 is the most recent day with complete data. We should discard the data after that date.

In [ ]:
# size of the dataset before applying the date filter
print(orders.shape)

In [ ]:
from datetime import date

max_date = 

# apply the date filter
orders = 

In [ ]:
# size of the dataset after applying the date filter
print(orders.shape)

In [ ]:
# check


There's a huge spike in the number of orders sometime around November/December 2017. Let's identify that specific day.

In [ ]:
# find the date associated with the maximum daily order volume



In [ ]:
# let's check the order_status for these orders



In [ ]:
# let's take a look at the order hour (hour of day) for these orders



In [ ]:
# let's plot it



This type of finding would be a good opportunity to talk with the business owners to find out the reason(s) behind this anomaly/spike. Here are some potential reasons for this spike: a big one-day sale event (Thanksgiving), big product launch, platform launch (e.g., a new marketplace) on that day.

### Is there a monthly seasonality in the order volumes?

In [ ]:
# extract year from the order date

orders['order_year'] = 

orders['order_year'].value_counts()

Note: `value_counts()` sorts the results by using the counts in a descending order.

In [ ]:
# extract month from the order date

orders['order_month'] = 

orders['order_month'].value_counts()

In [ ]:
# group by year and month to get the number of orders for each combination



In [ ]:
# reset index to make it a single-index (instead of multi-index) data frame



In [ ]:
# save the groupby results in a data frame

order_counts_by_month = 

In [ ]:
# plot the order volumes by year and month

sns.set(style='darkgrid')

plt.figure(figsize=[12, 9])

sns.lineplot()

plt.ylabel('Number of Orders', fontsize=14)
plt.xlabel('Order Purchase Date', fontsize=14);

The data from 2018 in incomplete (available until August), and only three months' worth of data is available from 2017. But based on the available data, there doesn't appear to be any strong seasonality effect over months. 

### Let's take a look at some customer-level patterns

The `customer_id` that's available in the `olist_orders_dataset` file is *not* unique for each customer. The `olist_customers_dataset` file contains the unique identifier for each customer.

In [ ]:
# read the file that contains the unique customer identifier

cust = pd.read_csv(cust_file)

cust.shape

In [ ]:
cust.head()

In [ ]:
# let's keep only the following two columns: customer_id, customer_unique_id

cust = 

Merge the orders dataframe with the customer dataframe to append `customer_unique_id` to the orders data.

In [ ]:
# merge orders and cust dataframes

orders = 

_`pandas`_ uses a similar structure like **SQL** for joining (merging) dataframes. Here's an analogous SQL query:

`CREATE TABLE orders as`

`SELECT a.*, b.*`

`FROM orders as a`

`JOIN cust as b`

`ON a.customer_id = b.customer_id;`

Note that an `inner` join is fine here because customers that are present in one dataframe are all also present in the other dataframe.

In [ ]:
# confirm how many unique customer IDs we had in the 'orders' dataset



In [ ]:
# number of orders per customer



Another approach, to get the order counts by customer, is to use `groupby()`.

In [ ]:
# number of orders per customer using groupby



Alternatively, use `orders.groupby('customer_unique_id')['order_id'].count()`.

In [ ]:
# save these counts (number of orders) in a new dataframe
cust_orders = orders.groupby('customer_unique_id').size().reset_index()

cust_orders.head()

In [ ]:
# rename the size/count column
cust_orders = 

cust_orders.head()

In [ ]:
# number of customers by their order volume

cust_orders.orders.value_counts(sort=False)

Most customers ordered only once :(

In [ ]:
# % of customers by their order volume

cust_orders['orders'].value_counts() / len(cust_orders)